In [11]:
import numpy as np
import matplotlib.pyplot as plt 
import os 
import cv2
import random
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_url = '/content/drive/MyDrive/Dataset'
categories = ["0","1","2","3","4","5","6","7","8","9"]
dataset = []
out = []
# store and normalize data 
for cat in categories :
    path = os.path.join(data_url,cat)
    for imgPath in os.listdir(path):
        image = cv2.imread(os.path.join(path,imgPath),cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (64,64))
        image = image.astype('float32')
        image /= 255
        dataset.append(image)
        out.append(categories.index(cat))
        #plt.imshow(image,cmap="gray")
        #plt.show()
        

In [6]:
testSize= len(dataset) * 0.2
x, x_valid, y, y_valid = train_test_split(dataset, out, test_size=0.15, random_state=42)


x = np.asarray(x)
y = np.asarray(y)
x_valid = np.asarray(x_valid)
y_valid = np.asarray(y_valid)


x = x.reshape(x.shape[0],64,64,1)
x_valid = x_valid.reshape(-1,64,64,1)


y = to_categorical(y, 10)
#y_test = to_categorical(y_test, 10)
y_valid = to_categorical(y_valid,10)

# print(y_train.shape)
# plt.imshow(x[9],cmap="gray")
# plt.show()


In [7]:
model = Sequential([
    Flatten(),
    Dense(128, activation='relu'), # hidden layer 1 with 128 neurons and input layer 
    Dense(64, activation='relu'),# hidden layer 2 with 64 neurons
    Dense(32, activation='relu'),  # hidden layer 3 with 32 neurons
    Dense(10, activation='sigmoid'),  # out layer 
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#hist = model.fit(x_train, y_train,batch_size=32, epochs=10,validation_data=(x_valid, y_valid))

In [8]:
# k-fold cross-validation during trianing
scores, histories = list(), list()
sum_ = 0.0
# prepare cross validation
K=4
kfold = KFold(K)

X_train, X_test,y_train, y_test = 0,0,0,0

# enumerate splits
for train_index, test_index in kfold.split(x):
    # select rows for train and test
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # fit model
    history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_valid, y_valid), verbose=0)
    # evaluate model
    _, acc = model.evaluate(x[test_index], y[test_index], verbose=0)
    # stores scores
    scores.append(acc)
    histories.append(history)
    sum_ += acc
scoreOfVal = sum_ / K

#Testing Accuracy
_, scoreOfTest = model.evaluate(x_test, y_test, batch_size=32, verbose=0)


In [12]:

print('accuracy for Validation = ', scoreOfVal)
print('accuracy for Testing = ', scoreOfTest)

y_pred = model.predict(x_test)
# print(y_pred)


y_pred_ =np.argmax(y_pred, axis=1)
y_test_ =np.argmax(y_test, axis=1)
# print(y_test)


print(classification_report(y_test_, y_pred_))
print(confusion_matrix(y_test_, y_pred_))

# # Precision= TP/(TP + FP) = Accuracy of positive predictions.

# # Recall = TP/(TP+FN) = Fraction of positives that were correctly identified.

# # F1 Score = 2*(Recall * Precision) / (Recall + Precision) = What percent of positive predictions were correct? 



accuracy for Validation =  0.6015981733798981
accuracy for Testing =  0.6598173379898071
14/14 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.94      0.82      0.88        39
           1       0.93      0.68      0.79        38
           2       0.40      0.95      0.56        38
           3       0.94      0.68      0.79        44
           4       0.46      0.40      0.43        40
           5       0.97      0.78      0.86        40
           6       0.46      0.70      0.55        47
           7       0.61      0.45      0.52        44
           8       0.69      0.58      0.63        59
           9       0.94      0.63      0.76        49

    accuracy                           0.66       438
   macro avg       0.73      0.67      0.68       438
weighted avg       0.73      0.66      0.67       438

[[32  0  0  0  2  0  3  1  1  0]
 [ 1 26  6  0  0  0  1  3  1  0]
 [ 0  1 36  0  0  0  0  0  1  0]
